In [7]:
from sklearn.model_selection import train_test_split
import pandas as pd


data = pd.read_csv("/Users/florianhaglsperger/Desktop/Coding/application_tracking/job_applicant_dataset.csv")
train_data, test_data = train_test_split(data, test_size=0.2, random_state=42)

with open("train_data.csv", "w") as train_file:
    train_data.to_csv(train_file, index=False)
with open("test_data.csv", "w") as test_file:
    test_data.to_csv(test_file, index=False)

In [19]:
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np
from transformers import GPT2Tokenizer
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader

class JobsDataset():
    def __init__(self, tokenizer, path):
        self.tokenizer = tokenizer

        self.data = pd.read_csv(path)
        self.job_description = self.data['Job Description'].tolist()
        self.resume = self.data['Resume'].tolist()
        self.age = self.data['Age'].tolist()
        self.best_match = self.data['Best Match'].tolist()

    def __len__(self):
        return len(self.job_description)

    def __getitem__(self, idx):
        description = self.job_description[idx]
        resume = self.resume[idx]
        age = self.age[idx]
        best_match = self.best_match[idx]

        description_encoding = self.tokenizer.encode(description)
        resume_encoding = self.tokenizer.encode(resume)

        return [
            [description_encoding, resume_encoding, age],
            [best_match]
        ]
    
def coallate_fn(batch):
    inputs, targets = zip(*batch)
    description_encoding, resume_encoding, age = zip(*inputs)
    best_match = [target[0] for target in targets]

    description_encoding = [torch.tensor(t) for t in description_encoding]
    resume_encoding = [torch.tensor(t) for t in resume_encoding]
    age = torch.tensor(age, dtype=torch.float32)
    best_match = torch.tensor(best_match, dtype=torch.float32)

    description_encoding = pad_sequence(description_encoding, batch_first=True, padding_value=0)
    resume_encoding = pad_sequence(resume_encoding, batch_first=True, padding_value=0)
    return [description_encoding, resume_encoding, age], best_match

dataset = JobsDataset(GPT2Tokenizer.from_pretrained('gpt2'), "train_data.csv")
train_loader = DataLoader(dataset, batch_size=32, shuffle=True, collate_fn=coallate_fn)

In [ ]:
class JobMatching(nn.Module):
    def __init__(self, vocab_size, embed_size, hidden_size = 256, output_size = 2):
        super(JobMatching, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embed_size)
        self.lstm = nn.LSTM(embed_size, hidden_size, batch_first=True)
        self.fc1 = nn.Linear(hidden_size, hidden_size //2)
        self.fc2 = nn.Linear(hidden_size //2, hidden_size //4)
        self.fc3 = nn.Linear(hidden_size //4, output_size)

    def forward(self, description, resume, age):
        

_IncompleteInputError: incomplete input (3865153847.py, line 9)